# Recommendation

Approche choisie pour créer un système de recommandation : Algorithme de clustering

In [2]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import json

Récupère les données des images favorites de l'utilisateur

In [108]:
# Charge les données du fichier JSON
with open("profil.json", "r", encoding="utf-8") as file:
    json_data = json.load(file)

# Extrait les données et les résultats
data = json_data["data"]  
result = json_data["resultats"]  

# Crée le DataFrame
dataframe = pd.DataFrame(data, columns=["orientation", "qualité", "animal", "nature", "luminosité"])

# Ajoute la colonne des résultats
dataframe['result'] = result

# Filtre les données pour ne garder que les images favorites
df_favorite = dataframe[dataframe['result'] == 'Favorite']

# Supprime la colonne 'result' après le filtrage
df_favorite = df_favorite.drop(columns=["result"])

# Convertit le DataFrame en une liste
favorite_list = df_favorite.values.tolist()

In [109]:
print("longueur :", len(df_favorite))
print(favorite_list)
print(df_favorite)

longueur : 5
[['Paysage', 'basse', 'Chien', 'en nature', 'sombre'], ['Paysage', 'moyenne', 'Chien', 'en nature', 'lumineux'], ['Paysage', 'haute', 'Chien', 'en nature', 'sombre'], ['Paysage', 'moyenne', 'Chien', 'en nature', 'sombre'], ['Paysage', 'basse', 'Unknown', 'pas de nature', 'sombre']]
   orientation  qualité   animal         nature luminosité
2      Paysage    basse    Chien      en nature     sombre
9      Paysage  moyenne    Chien      en nature   lumineux
14     Paysage    haute    Chien      en nature     sombre
16     Paysage  moyenne    Chien      en nature     sombre
18     Paysage    basse  Unknown  pas de nature     sombre


Récupère les données de toutes les images existantes et enlève les 20 images proposées à l'utilisateur

In [111]:
# Charge les données du fichier JSON
with open("profil.json", "r", encoding="utf-8") as file:
    json_data = json.load(file)

# Récupérer la liste des chemins des images
img_paths = json_data["images"]

In [116]:
# Charge les données du fichier JSON
with open("etiquetage.json", "r", encoding="utf-8") as file:
    json_data = json.load(file)

# Crée le DataFrame
dataframe = pd.DataFrame(json_data, columns=["nom_image", "orientation", "qualité", "animal", "nature", "luminosité"])

# Filtrer les données pour exclure les images spécifiées
dataframe_filtered = dataframe[~dataframe["nom_image"].isin(img_paths)]

# Supprime la colonne 'nom_image' après le filtrage
#dataframe_filtered = dataframe_filtered.drop(columns=["nom_image"])

# Convertit le DataFrame en une liste
data_list = dataframe_filtered.values.tolist()

In [117]:
print("longueur :", len(data_list))
print(data_list)
print(dataframe_filtered)

longueur : 96
[['1.jpg', 'Paysage', nan, 'Chien', nan, nan], ['10.jpg', 'Portrait', nan, 'Chat', nan, nan], ['100.jpg', 'Paysage', nan, 'Chien', nan, nan], ['101.png', 'Portrait', nan, 'Chat', nan, nan], ['102.jpg', 'Paysage', nan, 'Chien', nan, nan], ['103.jpg', 'Paysage', nan, 'Chien', nan, nan], ['105.jpg', 'Paysage', nan, 'Chien', nan, nan], ['106.jpg', 'Portrait', nan, 'Chat', nan, nan], ['107.jpg', 'Portrait', nan, 'Chat', nan, nan], ['108.jpg', 'Paysage', nan, 'Chat', nan, nan], ['109.jpg', 'Paysage', nan, 'Chat', nan, nan], ['11.jpg', 'Paysage', nan, 'Chat', nan, nan], ['110.jpg', 'Portrait', nan, 'Chat', nan, nan], ['111.jpg', 'Portrait', nan, 'Chien', nan, nan], ['112.jpg', 'Paysage', nan, 'Chat', nan, nan], ['113.jpg', 'Paysage', nan, 'Chat', nan, nan], ['114.jpg', 'Paysage', nan, 'Chien', nan, nan], ['115.jpg', 'Portrait', nan, 'Chat', nan, nan], ['116.jpg', 'Paysage', nan, 'Chien', nan, nan], ['12.JPG', 'Paysage', nan, 'Chat', nan, nan], ['13.jpg', 'Carré', nan, 'Chien', n

In [115]:
# Encode categorical features
label_encoders = [LabelEncoder() for _ in range(len(data_list[0]))]
encoded_data = []
for i, column in enumerate(zip(*data_list)):
    encoded_data.append(label_encoders[i].fit_transform(column))

X = list(zip(*encoded_data))  # Features

# Clustering
k = 2  # Number of clusters
kmeans = KMeans(n_clusters=k, n_init=10)
kmeans.fit(X)
clusters = kmeans.labels_

# Add cluster labels to the original data
data_with_clusters = pd.DataFrame(data_list, columns=["orientation", "qualité", "animal", "nature", "luminosité"])
data_with_clusters["Cluster"] = clusters

# Recommendation function
def recommend_items(cluster, data_with_clusters):
    items_in_cluster = data_with_clusters[data_with_clusters["Cluster"] == cluster]
    recommended_items = items_in_cluster.sample(n=4)  # Sample 3 items from the cluster
    return recommended_items

user_interaction = favorite_list[0]  # Assuming user interacted with this item
encoded_interaction = [label_encoders[i].transform([val])[0] for i, val in enumerate(user_interaction)]
cluster = kmeans.predict([encoded_interaction])[0]
recommendations = recommend_items(cluster, data_with_clusters)
print("Recommended items:")
print(recommendations)

Recommended items:
   orientation qualité animal         nature luminosité  Cluster
44     Paysage   basse  Chien      en nature     sombre        0
86     Paysage   basse  Chien  pas de nature     sombre        0
54     Paysage   basse   Chat  pas de nature   lumineux        0
46    Portrait   basse  Chien      en nature   lumineux        0
